In [2]:
import argparse
import math
import os
import random
import sys
import time
from copy import deepcopy
from datetime import datetime
from pathlib import Path

import numpy as np
import torch
import torch.distributed as dist
import torch.nn as nn
import yaml
from torch.cuda import amp
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.optim import SGD, Adam, AdamW, lr_scheduler
from tqdm import tqdm
import pandas as pd

import val  # for end-of-epoch mAP
from models.experimental import attempt_load
from models.yolo import Model
from utils.autoanchor import check_anchors
from utils.autobatch import check_train_batch_size
from utils.callbacks import Callbacks
from utils.datasets import create_dataloader
from utils.downloads import attempt_download
from utils.general import (LOGGER, check_dataset, check_file, check_git_status, check_img_size, check_requirements,
                           check_suffix, check_yaml, colorstr, get_latest_run, increment_path, init_seeds,
                           intersect_dicts, labels_to_class_weights, labels_to_image_weights, methods, one_cycle,
                           print_args, print_mutation, strip_optimizer)
from utils.loggers import Loggers
from utils.loggers.wandb.wandb_utils import check_wandb_resume
from utils.loss import ComputeLoss
from utils.metrics import fitness
from utils.plots import plot_evolve, plot_labels
from utils.torch_utils import EarlyStopping, ModelEMA, de_parallel, select_device, torch_distributed_zero_first

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: boostcv16 (use `wandb login --relogin` to force relogin)


In [6]:
PROJECT_NAME = 'Inseo_Lee_YOLO'
EXPERIMENT_NAME = 'exp2'
DATA_ROOT = '../dataset'
TEST_DATASET_PATH = os.path.join(DATA_ROOT, 'test')

INFERENCE_SIZE = 1024
AUGMENT = True


IMAGES = []
for image_name in os.listdir(TEST_DATASET_PATH):
    if not image_name.startswith('._'):
        IMAGES.append(os.path.join(TEST_DATASET_PATH, image_name))

IMAGES.sort()

model = torch.hub.load(
    repo_or_dir = './',
    model = 'custom',
    path = f'../{PROJECT_NAME}/{EXPERIMENT_NAME}/weights/best.pt',
    source='local')

model.conf = 0.001  # NMS confidence threshold
model.iou = 0.6  # NMS IoU threshold
# model.agnostic = False  # NMS class-agnostic
# model.multi_label = False  # NMS multiple labels per box
# model.classes = None  # (optional list) filter by class, i.e. = [0, 15, 16] for COCO persons, cats and dogs
# model.max_det = 1000  # maximum number of detections per image
# model.amp = False  # Automatic Mixed Precision (AMP) inference
IMAGES[0]



YOLOv5 🚀 7808ed6 torch 1.10.0+cu102 CUDA:0 (Tesla V100-PCIE-32GB, 32510MiB)

Fusing layers... 
Model summary: 574 layers, 140057380 parameters, 0 gradients, 208.3 GFLOPs
Adding AutoShape... 


'../dataset/test/0000.jpg'

In [7]:
prediction_strings = []
file_names = []

for i in tqdm(range(len(IMAGES))):
    prediction_string = ''
    
    output = model(IMAGES[i], size=INFERENCE_SIZE, augment=AUGMENT)
    for bbox in output.pandas().xyxy[0].values:
        xmin, ymin, xmax, ymax, conf, cls, _ = bbox
        prediction_string += f'{cls} {conf} {xmin} {ymin} {xmax} {ymax} '
    
    path_split = IMAGES[i].split('/')
    image_path = f'{path_split[-2]}/{path_split[-1]}'

    prediction_strings.append(prediction_string)
    file_names.append(image_path)

submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.head()

100%|██████████| 4871/4871 [11:03<00:00,  7.34it/s]


,PredictionString,image_id
0,7 0.9869502186775208 602.1071166992188 518.241...,test/0000.jpg
1,5 0.9805595278739929 346.0233459472656 250.407...,test/0001.jpg
2,1 0.9810634851455688 776.2323608398438 414.580...,test/0002.jpg
3,9 0.9586802124977112 143.323974609375 261.0804...,test/0003.jpg
4,0 0.9656744003295898 193.46148681640625 252.17...,test/0004.jpg


In [11]:
prediction_strings[0]

'7 0.9911515116691589 602.6592407226562 518.3744506835938 957.2759399414062 1022.9487915039062 7 0.9888994693756104 216.6562042236328 51.11479187011719 456.41973876953125 471.2139892578125 7 0.9844570755958557 447.51336669921875 600.78369140625 647.1171264648438 875.4075927734375 7 0.9827660918235779 119.17108917236328 418.4630126953125 332.15460205078125 712.138916015625 7 0.9764297008514404 650.8900756835938 334.5909423828125 877.3859252929688 601.1932983398438 7 0.9756133556365967 390.62939453125 190.2154083251953 623.306884765625 543.3621215820312 5 0.9587175846099854 243.43389892578125 685.1572875976562 301.1790771484375 748.4664916992188 7 0.956315815448761 547.1837158203125 337.4664306640625 705.528076171875 603.2843627929688 1 0.9525213837623596 567.4407958984375 101.11203002929688 766.6722412109375 358.4510803222656 5 0.9481369256973267 342.8555603027344 643.3126220703125 395.7364196777344 725.853515625 7 0.9452932476997375 238.52243041992188 403.49517822265625 431.57473754882

In [9]:
submission.to_csv(os.path.join(f'../{PROJECT_NAME}/{EXPERIMENT_NAME}', 'submission_exp2_0.01.csv'), index=None)

In [ ]:
results.pandas()
results.pandas().xyxy[0]

In [ ]:
prediction_strings = []
file_names = []